<a href="https://colab.research.google.com/github/poliakovfm/itmo_transcriptomics_homeworks/blob/main/HW1_Trans_Meta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 1 for "Transcriptomics and metagenomics" course


#### [Assignment description](https://docs.google.com/document/d/1B4RiAuhA68TXJIHcmr7iTW-8G0tbsVeoYFQLSXbjBDY/edit)

# 0. Installing required packages and tools

#### 0.1. Installing Biopython

In [1]:
!pip install biopython

#### 0.2. Installing QUAST

In [2]:
!git clone https://github.com/ablab/quast.git

fatal: destination path 'quast' already exists and is not an empty directory.


#### 0.3. Loading required libraries

In [3]:
import pandas as pd
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Initial checking of the sequencing data

### 1.1. Downloading and checking the data assembled with SPAdes assembler

In [5]:
k = 0

for seq_record in SeqIO.parse("/content/drive/MyDrive/HW1_files/spades_scaffolds.fasta", "fasta"):
  print(seq_record.id)
  print(repr(seq_record.seq))
  print(len(seq_record))
  k += 1

  if k > 5:
    break


NODE_1_length_29907_cov_150.822528
Seq('GTGTTTGATTTTTTTTTTTTTTTTTTTTTTTTGTCATTCTCCTAAGAAGCTATT...ATC')
29907
NODE_2_length_8134_cov_6.182822
Seq('CCCTAGTTGCTGAACCAGCGATAACTCCTGTCCCTTTTGAAGCTGGTTTCAATA...GTC')
8134
NODE_3_length_5584_cov_13759.613946
Seq('AAGCCTTCAAGAAGGTGATAAGCAGGAGAAACATACGAAGGCGCATAACGATAC...CTT')
5584
NODE_4_length_5116_cov_18.119468
Seq('TCCGTCAACTCAAGTAACTTATCAACCAAAAGGTCTTTCAAGTCATTTACCTTT...TTT')
5116
NODE_5_length_4814_cov_12.303145
Seq('CCCCATTTTGAATTATGAATTGTGAATTTTGAATTAAAAAAAGGTATAGACTCT...CCC')
4814
NODE_6_length_4266_cov_43.850322
Seq('AGCCACTAAAGTGGTGTTATAGCCCTTTTGTACATGTATAACAATTAAATTAAT...AGC')
4266


### 1.2. Running QUAST tool to get basic statistics of our assemblies

In [6]:
# first we pass the path to quast.py, then the output folder and finally the path to .fasta file

! python /content/quast/quast.py -o quast_results /content/drive/MyDrive/HW1_files/spades_scaffolds.fasta

/content/quast/quast.py -o quast_results /content/drive/MyDrive/HW1_files/spades_scaffolds.fasta

Version: 5.2.0

System information:
  OS: Linux-5.15.120+-x86_64-with-glibc2.35 (linux_64)
  Python version: 3.10.10
  CPUs number: 2

Started: 2023-10-03 09:26:44

Logging to /content/quast_results/quast.log
NOTICE: Output directory already exists and looks like a QUAST output dir. Existing results can be reused (e.g. previously generated alignments)!
NOTICE: Maximum number of threads is set to 1 (use --threads option to set it manually)

CWD: /content
Main parameters: 
  MODE: default, threads: 1, min contig length: 500, min alignment length: 65, min alignment IDY: 95.0, \
  ambiguity: one, min local misassembly length: 200, min extensive misassembly length: 1000


Contigs:
  Pre-processing...
  /content/drive/MyDrive/HW1_files/spades_scaffolds.fasta ==> spades_scaffolds

2023-10-03 09:26:46
Running Basic statistics processor...
  Contig files: 
    spades_scaffolds
  Calculating N50 and

In [7]:
# now we can check the resulting statistics,
# including the number of contigs >= 1000 bp (666) and the size of the largest one (29907 bp)

quast_results = pd.read_csv("/content/quast_results/report.tsv", sep='\t')
quast_results

,Assembly,spades_scaffolds
0,# contigs (>= 0 bp),111219.00
1,# contigs (>= 1000 bp),666.00
2,# contigs (>= 5000 bp),4.00
3,# contigs (>= 10000 bp),1.00
4,# contigs (>= 25000 bp),1.00
5,# contigs (>= 50000 bp),0.00
6,Total length (>= 0 bp),27362468.00
7,Total length (>= 1000 bp),1042805.00
8,Total length (>= 5000 bp),48741.00
9,Total length (>= 10000 bp),29907.00


In [8]:
# alternative way to find the number of contigs >= 1000 bp and the size of the largest one

largest_contig = 0
contigs_larger_than_1000 = 0

for seq_record in SeqIO.parse("/content/drive/MyDrive/HW1_files/spades_scaffolds.fasta", "fasta"):

  if len(seq_record) >= 1000:
    contigs_larger_than_1000 += 1

  if len(seq_record) > largest_contig:
    largest_contig = len(seq_record)


print("Number of contigs >= 1000 bp:", contigs_larger_than_1000)
print("Size of the largest contig:", largest_contig)

Number of contigs >= 1000 bp: 666
Size of the largest contig: 29907


# 2. Finding contigs of interest

#### For this step we will use the data produced by ViralVerify program.
#### We will use two files: one file with ViralVerify results and one .fasta file containing only viral contigs.

In [9]:
# let's have a look at viralverify output

viralverify_output = pd.read_csv("/content/drive/MyDrive/HW1_files/spades_contig_result_table.csv")

viralverify_output.head(6)

,Сontig_name,Prediction_result,Length,Circular,Score,HMMs
0,NODE_1_length_29907_cov_150.822528,Virus,29907,-,185.77,Sars6 Corona_S2 Spike_rec_bind APA3_viroporin ...
1,NODE_2_length_8134_cov_6.182822,Chromosome,8134,-,-102.81,Ribosomal_L4 Ribosomal_L16 Ribosomal_S3_C KH_2...
2,NODE_3_length_5584_cov_13759.613946,Virus,5584,-,14.37,Microvir_J Phage_F Microvir_H Phage_G Phage_C ...
3,NODE_4_length_5116_cov_18.119468,Chromosome,5116,-,-21.66,Ribosomal_L1 Ribosomal_L10 Ribosomal_L12 Ribos...
4,NODE_5_length_4266_cov_43.850322,Chromosome,4266,-,-9.32,Ribosomal_S21 Phage_integrase Phage_int_SAM_1 ...
5,NODE_6_length_4083_cov_14.312531,Chromosome,4083,-,-16.18,ATP-synt OSCP ATP-synt_ab ATP-synt_ab_C ATP-sy...


In [10]:
# printing basic stats

print("Total number of contigs:", len(viralverify_output))
print("Number of viral contigs:", len(viralverify_output[viralverify_output['Prediction_result'] == 'Virus']))

Total number of contigs: 111270
Number of viral contigs: 194


# 3. BLASTing the contig of interest

##### After looking at the contig table info, it became obvious that the first contig is the most interesting for our task, as its length is of orders of magnitude higher than others' ones

In [11]:
# finding the id of largest contig in our file

contig_name = viralverify_output[viralverify_output["Length"] == viralverify_output["Length"].max()].iat[0,0]
contig_name

'NODE_1_length_29907_cov_150.822528'

In [12]:
# finding the nt sequence of the largest contig

for seq_record in SeqIO.parse("/content/drive/MyDrive/HW1_files/spades_scaffolds.fasta", "fasta"):
  if seq_record.id == contig_name:
    contig_seq = seq_record.seq
    break

contig_seq = str(contig_seq)
print(contig_seq)

...


In [13]:
# running BLAST


fasta_sequence = f">Viral_contig\n{contig_seq}"


# creating df to store BLAST results
columns = ["Query Sequence", "Hit Title", "Hit Description", "Hit Accession",
           "Species", "E-Value", "Bit-Score", "Hit sequence"]
df = pd.DataFrame(columns=columns)


# defining Entrez query to date range and to viral genomes
entrez_query = "1900/01/01:2020/01/01[PDAT] AND txid10239[Organism:exp]"


# performing BLAST search
result_handle = NCBIWWW.qblast("blastn", "nt", fasta_sequence, entrez_query=entrez_query)


# parsing the output
blast_record = NCBIXML.read(result_handle)

# selecting the top hit
if blast_record.alignments:
    top_hit = blast_record.alignments[0]
    hsp = top_hit.hsps[0]

    # splitting the title to extract species information
    title_parts = top_hit.title.split("|")
    species = title_parts[-1] if len(title_parts) > 1 else "Unknown"

    # creating a new row in df
    row = {
        "Query Sequence": fasta_sequence,
        "Hit Title": top_hit.title,
        "Hit Description": top_hit.hit_def,
        "Hit Accession": top_hit.accession,
        "Species": species,
        "E-Value": hsp.expect,
        "Bit-Score": hsp.bits,
        "Hit Sequence": hsp.sbjct,
    }

    # and adding it
    df = df.append(row, ignore_index=True)


result_handle.close()

df

<ipython-input-13-14df5b35cbbc>:46: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


,Query Sequence,Hit Title,Hit Description,Hit Accession,Species,E-Value,Bit-Score,Hit sequence,Hit Sequence
0,>Viral_contig\nGTGTTTGATTTTTTTTTTTTTTTTTTTTTTT...,gi|1369125417|gb|MG772933.1| Bat SARS-like cor...,Bat SARS-like coronavirus isolate bat-SL-CoVZC...,MG772933,Bat SARS-like coronavirus isolate bat-SL-CoVZ...,0.0,28811.9,NaN,TTCCAAGCTATGACACAACCTGTAAAATCATCAGGTAATTTATAAT...


In [14]:
print('Full BLAST hit description:', df.iloc[0].loc['Hit Description'])
print('BLAST hit species:', df.iloc[0].loc['Species'])

Full BLAST hit description: Bat SARS-like coronavirus isolate bat-SL-CoVZC45, complete genome
BLAST hit species:  Bat SARS-like coronavirus isolate bat-SL-CoVZC45, complete genome
